# Caching

In [7]:
#default_exp caching
#export
import pathlib
import pickle

from reseng.config import Paths

Module "reseng.config" read config from "/home/babkin/rurec/submodules/reseng/settings.ini".


In [9]:
#export
def simplecache(path):
    def wrapper(func):
        def wrapped(*args, **kwargs):
            p = pathlib.Path(path)
            if p.exists():
                print(f'Reading {func.__name__}() cached result from "{path}".')
                return pickle.load(p.open('rb'))
            else:
                res = func(*args, **kwargs)
                print(f'Writing {func.__name__}() result to cache at "{path}".')
                pickle.dump(res, p.open('wb'), protocol=5)
                return res
        return wrapped
    return wrapper

In [11]:
try:
    p = Paths.locate_root_path()/'tmp/test.pkl'
    @simplecache(p)
    def test_simplecache():
        print('--> calculating')
        return 1
    
    print('calculate')
    assert test_simplecache() == 1
    print('load cache')
    assert test_simplecache() == 1
    p.unlink()
    print('calculate')
    assert test_simplecache() == 1
finally:
    p.unlink()

calculate
--> calculating
Writing test_simplecache() result to cache at "/home/babkin/rurec/submodules/reseng/tmp/test.pkl".
load cache
Reading test_simplecache() cached result from "/home/babkin/rurec/submodules/reseng/tmp/test.pkl".
calculate
--> calculating
Writing test_simplecache() result to cache at "/home/babkin/rurec/submodules/reseng/tmp/test.pkl".
